This file makes and pickles q data as well as case study images. Also reads in fits data.


Dependencies...

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('mode.chained_assignment', None)
warnings.simplefilter("ignore", category=RuntimeWarning)
import re
import pickle
import struct
from datetime import datetime, timedelta
from os import fsdecode
import pathlib

import matplotlib
import matplotlib.dates as mdates
import matplotlib.gridspec as gs
import matplotlib.pyplot as plt
import numpy as np
import spiceypy as spice
import pandas as pd
import pycwt as wavelet
import scipy.integrate as integrate
import scipy.signal as signal
from matplotlib.colors import LogNorm
from mpl_toolkits.axes_grid1.inset_locator import inset_axes
from mpl_toolkits.axes_grid1 import make_axes_locatable
from spiceypy.spiceypy import xf2eul
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import math as m

import random
import os
import math


Classes from juno_classes.py pulled here to alter them to fit my needs...

In [ ]:
class PlotClass:
    def __init__(self, axes, xlabel=None, ylabel=None, title=None):
        self.axes = axes
        self.xlabel = xlabel
        self.ylabel = ylabel
        self.title = title

    def xaxis_datetime_tick_labels(self, x_ticks_labeled,):
        
        locator = mdates.AutoDateLocator(minticks=5, maxticks=20)
        formatter = mdates.ConciseDateFormatter(locator)
        self.axes.xaxis.set_major_locator(locator)
        self.axes.xaxis.set_major_formatter(formatter)
        if not x_ticks_labeled:
            self.axes.set_xticklabels([])

    def plot(self, x, y, magnitude=False, data_labels=None, xlabel=None, ylabel=None,
                         title=None, **kwargs):

        if (np.ndim(y) == 1) & (np.ndim(x) == 1):
            self.axes.plot(x, y, **kwargs)
        elif (np.ndim(y) != 1) & (np.ndim(x) != 1):
            self.axes.plot(np.transpose(x), np.transpose(y), **kwargs)
        else:
            if data_labels is None:
                self.axes.plot(x, np.transpose(y), **kwargs)
            else:
                for i in range(len(y)):
                    self.axes.plot(x, y[i], label=data_labels[i], **kwargs)
        
        if magnitude:
            mag = np.array([np.sqrt(np.sum(np.power(i, 2))) for i in np.transpose(y)])
            self.axes.plot(x, mag, label='Magnitude', color='black', **kwargs)
            self.axes.plot(x, -mag, color='black', **kwargs)

        if magnitude and data_labels != None:
            self.axes.legend(loc='upper left')
        self.axes.set_xlabel(self.xlabel)
        self.axes.set_ylabel(self.ylabel)
        self.axes.set_title(self.title)

    def colormesh(self, x, y, data, color_bar=True, xlabel=None, ylabel=None, title=None, **kwargs):

        cwt = self.axes.pcolormesh(x, y, data, shading='auto', **kwargs)
        #cwt = self.axes.pcolormesh(x, y, data)
        self.axes.set_title(title)
        self.axes.set_ylim(y[0], y[-1])
        self.axes.set_ylabel(ylabel)
        self.axes.set_xlabel(xlabel)
        self.axes.set_xlim(x[0], x[-1])
        if color_bar:
            divider = make_axes_locatable(self.axes)
            cax = divider.append_axes("right", '2%', pad='2%')
            cbr = plt.colorbar(cwt, cax=cax)
            cbr.set_label('Magnitude', rotation=270, labelpad=5, size=8)


class CWTData:
    def __init__(self, datetime_series, signal, dt, min_freq=None, max_freq=None,
                 wave_resolution=6, mother=wavelet.Morlet):
        """Class for calculating, manipulating, and plotting a continuous wavelet
        analysis of a signal.

        Parameters
        ----------
        datetime_series : array of datetime64[ns] data
            Array of datetime variables accompanying the signal data
        signal : array
            Signal to be analyzed
        dt : int
            Time in seconds between data samples in the signal
        min_freq : float
            Lowest frequency in the frequency range to calculate the cwt in.
            Leave as None to calculate over whole range.
        max_freq : float
            Highest frequency in the frequency range to calculate the cwt in.
            Leave as None to calculate over whole range.
        wave_resolution : int, optional
            Resolution to be used in the wavelet packet, by default 6
        mother : pycwt mother wavelet class, optional
            Wavelet class from the pycwt module, by default pycwt.Morlet

        """        
        self.time_series = datetime_series
        self.data = signal
        self.wave_resolution = wave_resolution
        self.dt = dt
        self.min_freq = min_freq
        self.max_freq = max_freq
        self.mother = mother
        self.peaks_found = False
        self._get_cwt_matrix()

    def _get_cwt_matrix(self):

        N = len(self.data)
        Fs = 1 / self.dt
        f = np.arange(0, N / 2 + 1) * Fs / N
        #f = np.linspace(0, N / 2 + 1, num=100) * Fs / N

        if self.min_freq is None:
            min_index = 1
        else:
            min_index = min(range(len(f)), key=lambda i: abs(f[i] - self.min_freq)) - 1
        if self.max_freq is None:
            max_index = len(self.data)
        else:
            max_index = min(range(len(f)), key=lambda i: abs(f[i] - self.max_freq))
        f = f[min_index: max_index]
        wave, self.scales, self.freqs, self.coi, fft, fftfreqs = wavelet.cwt(
            self.data, self.dt, self.mother(self.wave_resolution), freqs=f
        )
        #dj = 1 / 12
        #s0 = 2 * self.dt
        #J = 7 / dj
        #wave, scales, self.freqs, self.coi, fft, fftfreqs = wavelet.cwt(self.data, self.dt, dj, s0, J,
        #                                              self.mother(self.wave_resolution))
        self.power = np.abs(wave)**2
        self.coi = self.coi**-1

    def remove_coi(self):
        # Removes the data affected by the cone of influence
        for i, col in enumerate(self.power.T):
            col_num = len(col) - i
            coi_start_index = min(range(len(self.freqs)),
                                  key=lambda i: abs(self.freqs[i] - self.coi[col_num]))
            self.power[:coi_start_index, col_num] = np.zeros(coi_start_index)        

            
    def cwt_plot(self, axes, mark_coi=True, remove_coi=True, title=None, colorbar=True,
                 xlabel=None, time_per_major='12H', time_per_minor='1H',
                 tick_labels_format='%m-%d %H', x_ticks_labeled=True, **kwargs):

        if remove_coi:
            self.remove_coi()
        
        vmin = np.percentile(np.nan_to_num(self.power), 10)
        vmax = np.max(np.nan_to_num(self.power))
        t = mdates.date2num(self.time_series)    
        orbitsData = ['2016-07-31T19:46:02',
                  '2016-09-23T03:44:48',
                  '2016-11-15T05:36:45',
                  '2017-01-07T03:11:30',
                  '2017-02-28T22:55:48',
                  '2017-04-22T19:14:57',
                  '2017-06-14T15:58:35',
                  '2017-08-06T11:44:04',
                  '2017-09-28T07:51:01',
                  '2017-11-20T05:57:23',
                  '2018-01-12T03:52:42',
                  '2018-03-05T23:55:41',
                  '2018-04-27T19:36:40',  
                  '2018-06-19T17:30:40',
                  '2018-08-11T15:18:43',
                  '2018-10-03T10:58:52',   
                  '2018-11-25T07:01:26',
                  '2019-01-17T05:19:21',   
                  '2019-03-11T02:48:11',   
                  '2019-05-02T22:18:47',   
                  '2019-06-24T18:01:57',   
                  '2019-08-16T16:01:52',   
                  '2019-10-08T12:52:15',   
                  '2019-11-30T07:39:10',
                  '2020-01-22T05:44:55',   
                  '2020-03-15T03:44:40',   
                  '2020-05-07T00:16:41',   
                  '2020-06-28T20:24:51',   
                  '2020-08-20T16:08:49',   
                  '2020-10-12T14:05:43',
                  '2020-12-04T11:37:23',   
                  '2021-01-26T07:36:06',
                  '2021-03-20 08:39:35',
                  '2021-05-12 15:29:09']  
    
        plot_class = PlotClass(axes)
        plot_class.colormesh(t, self.freqs, self.power, ylabel='Frequency (Hz)', xlabel=xlabel,
                             title=title, color_bar=colorbar, norm=LogNorm(vmin=10**-4, vmax=10**1.2),
                             cmap='jet', **kwargs)
        if mark_coi:
            plot_class.plot(self.time_series, self.coi, linestyle='--', color='black')
        axes.set_yscale('log')
        plot_class.xaxis_datetime_tick_labels(x_ticks_labeled)
        

    def peaks_plot(self, axes, plot_title=False, xlabel=True,
                   time_per_major='12H', time_per_minor='1H',
                   tick_labels_format='%m-%d %H', x_ticks_labeled=True, **kwargs):

        if not self.peaks_found:
            self._peak_finding()

        t = mdates.date2num(self.time_series)
        plot_class = PlotClass(axes)
        try:
            plot_class.colormesh(t, self.freqs, self.power, xlabel=xlabel, title=plot_title,
                            norm=LogNorm(),cmap='jet', **kwargs)
        except:
            plot_class.colormesh(t, self.freqs, self.power, xlabel=xlabel, title=plot_title,
                            cmap='jet', **kwargs)
        axes.set_yscale('log')
        plot_class.xaxis_datetime_tick_labels(x_ticks_labeled)

    def peaks_hist(self, axes, min_frequency=1/(20*60*60), max_frequency=1/(60*60),
                   freq_per_bin=1, x_units='min'):

        if not self.peaks_found:
            self._peak_finding(min_frequency, max_frequency)

        units_switch = {'sec': 1,
                        'min': 60,
                        'hour': 3600,
                        'day': 86400}
        bin_num = round(len(self.peak_freq_range) / freq_per_bin)
        axes.hist(self.freq_peaks_hist, bins=bin_num)
        axes.tick_params(axis='x', labelsize='small')
        axes.set_xticks(np.linspace(min_frequency, max_frequency, 10))
        axes.set_xticklabels(np.round(
            1/(np.linspace(min_frequency, max_frequency, 10) * units_switch[x_units]), 1
            ))
        axes.set_xlabel(f'Time ({x_units})')
        axes.set_ylabel('Weighted Peaks')
        axes.set_yscale('log')

    def __psd(self):
        freq = self.freqs
        cwt_power = self.power
        self.psd = np.zeros(len(self.freqs))
        for i in range(0, len(freq)):
            self.psd[i] = (2/len(freq))*(sum(cwt_power[i, :]))
        return self.psd

    
    def psd_plot(self, axes, x_units, ylabel=None,
                 title=None, **kwargs):
        
        self.__psd()
        
        plot_class = PlotClass(axes, f'Time({x_units})', ylabel, title)
        plot_class.plot(self.freqs, self.psd, xlabel=f'Time ({x_units})', ylabel='Power',
                        color='blue', **kwargs)
        axes.set_yscale('log')
        
        units_switch = {'sec': 1, 'min': 60, 'hour': 3600, 'day': 86400}
        axes.set_xticks(np.linspace(self.freqs[0], self.freqs[-1], 10))
        axes.set_xticklabels(np.round(
                    1/(np.linspace(self.freqs[0], self.freqs[-1], 10) 
                       * units_switch[x_units.lower()]), 1))

    def calc_freq_bandpower(self):
        
        self.remove_coi()
        bandpower = np.zeros(len(self.time_series), dtype='float')
        for i in range(len(bandpower)):
            avg = integrate.trapz(self.power[:, i], self.freqs)
            bandpower[i] = avg

        return bandpower
            


Q calculation and case study images making within the q calculation. There are also several functions that are used in the calculation...

In [ ]:
#Q attempt to make desired psd and time series ...


#These are all the dependent functions ...
#q calculation is inlcuded to have case study applications

def _temp_dist(RRR):    
    a1 = -0.116
    a2= 2.14
    a3= -2.05
    a4= 0.491
    a5= 0.126
    amu=22
    rrr=np.log10(RRR/6)
    h=a1+a2*rrr+a3*rrr**2.+a4*rrr**3.+a5*rrr**4.

    H=10.**h

    T=amu*(H/0.64)**2.

    T_kelvin = T*11606
    return T_kelvin

def _v_therm(T_kelvin, mi):
    kB = 1.38e-23
    vthermal = np.sqrt(2*kB * T_kelvin/mi) # E_perp = 1/2m(v_perp_therm)^2 = kT
    return vthermal

def _gyro(bx, by, bz, m, z, q):
    """finds a gyrofreqency given magnetosphere properties.  \n All inputs must be in
    fundamental units (T, m, etc.) \n Returns scalar qyrofrequency corresponding to given
    range of B"""
    mean_b = np.mean(np.sqrt(bx**2 + by**2 + bz**2))
    gyrofreq = (z*q/m)*(mean_b)/(2*np.pi)
    return gyrofreq

def _rho_i(vth, gyrofreq):
    rho_i = vth/(2*np.pi*(gyrofreq/z))
    return rho_i

def kperpgiver(v_rel, freq):
    kperp = (2*np.pi*freq)/(v_rel*np.sin(np.pi/2)) 
    return kperp


def _psd(time_array,cwt_power,freq,fs):
    """Finds PSD as per Tao et. al. 2015 given a morlet wavelet transform, frequency 
    range, the signal, and sampling frequency. \n Outputs an array of length of signal"""
    psd = np.zeros(len(freq))
    for i in range(0, len(freq)):
        if len(np.where(cwt_power[i,:]>0)[0]) > 0:
            psd[i] = (2/len(np.where(cwt_power[i,:]>0)[0]))*(sum(cwt_power[i][:]))
    return psd

def get_break_coeff_and_slopes(theo_break, minf, maxf, cwt_freqs, cwt_psd, gyro, pad, thresh, ftb, acceptable_freqs_min, acceptable_freqs_max):
    guess = gyro
    minmin = minf
    maxmax = maxf
    theoretical_break = ftb
    keep_index = 0

    break_points = np.arange(minf, maxf, .01)
    x = (cwt_freqs[np.where(cwt_psd >0)])
    y = (cwt_psd[np.where(cwt_psd >0)])
    # Define target slope ranges
    target_left_min = (-5./3.) - thresh # Example minimum slope for left line
    target_left_max = (-5./3.) + thresh# Example maximum slope for left line
    target_right_min = (-7./3.) - thresh# Example minimum slope for right line
    target_right_max = (-7./3.) + thresh # Example maximum slope for right line

    # Prepare to store counts of slopes within the target ranges
    break_point_counts = []
    
    #do it normal
    # Iterate over possible break points
    
    if theo_break:
        best_break_point = ftb

        if (theoretical_break <= acceptable_freqs_min):
        #do what needs to be done if the theo break is less than the acceptable min, make all kaw
            left_mask =    0
            right_mask = (x > minmin) & (x<maxmax)
            
            model_left = 0
            model_right = LinearRegression().fit(np.log10(x[right_mask]).reshape(-1, 1), np.log10(y[right_mask]))
            
            slope_left = 0
            slope_right = model_right.coef_[0]
            
            

            if (slope_right < (-7./3.) - thresh*2) or (slope_right > (-7./3.) + thresh*2):
                keep_index = 1
                #if (slope_right > 1.25) or (slope_right < -3.5):
                #    slope_left = 0
                #    slope_right = 0

            

        
        elif (theoretical_break >= acceptable_freqs_max):
            #do what needs to be done if the theo break is greater than the acceptable max, make all mhd
            left_mask =    (x > minmin) & (x<maxmax)
            right_mask = 0
            
            model_left = LinearRegression().fit(np.log10(x[left_mask]).reshape(-1, 1), np.log10(y[left_mask]))
            model_right = 0
            
            slope_left = model_left.coef_[0]
            slope_right = 0
            
            
  
   
            if (slope_left < (-5./3.) - thresh) or (slope_left > (-5./3.) + thresh):
                keep_index = 1
                #if (slope_left > 0.25) or (slope_left < -3.1):
                #    slope_left = 0
                #    slope_right = 0
            
        else: 
        
            break_point = theoretical_break
            left_mask =    (x < break_point/pad)  & (x > minmin)
            right_mask = (x > break_point*pad) & (x<maxmax)
            
  
            

            model_left = LinearRegression().fit(np.log10(x[left_mask]).reshape(-1, 1), np.log10(y[left_mask]))

            model_right = LinearRegression().fit(np.log10(x[right_mask]).reshape(-1, 1), np.log10(y[right_mask]))
            
        
        
            slope_left = model_left.coef_[0]
            slope_right = model_right.coef_[0]
            

            
            if (slope_left < (-5./3.) - thresh) or (slope_left > (-5./3.) + thresh):
                keep_index = 1
         #       if (slope_left > 0.25) or (slope_left < -3.1):
          #          slope_left = 0
          #          slope_right = 0
            elif (slope_right < (-7./3.) - thresh*2) or (slope_right > (-7./3.) + thresh*2):
                keep_index = 1
           #     if (slope_right > 1.25) or (slope_right < -3.5):
           #         slope_left = 0
           #         slope_right = 0
           # else:
            best_break_point = break_point
        

        
    if not theo_break:
        for break_point in break_points:
        # Split data into two segments
            left_mask =    (x < break_point/pad)  & (x > minmin)
            right_mask = (x > break_point*pad) & (x<maxmax)

            if np.sum(left_mask) > 0 and np.sum(right_mask) > 0:
            # Fit lines to both segments
                model_left = LinearRegression().fit(np.log10(x[left_mask]).reshape(-1, 1), np.log10(y[left_mask]))
                model_right = LinearRegression().fit(np.log10(x[right_mask]).reshape(-1, 1), np.log10(y[right_mask]))
            
                slope_left = model_left.coef_[0]
                slope_right = model_right.coef_[0]

            # Count slopes within the target range
                if (target_left_min <= slope_left) and (slope_left<= target_left_max):
                    count_left = 2
                else: 
                    count_left = 0
                if (target_right_min <= slope_right) and (slope_right <= target_right_max):
                    count_right = 1
                else:
                    count_right = 0 
            
                total_count = count_left + count_right
            
            # Store the break point and total count
                break_point_counts.append((break_point, total_count))

        # Convert counts to an array for easier analysis
        break_point_counts = np.array(break_point_counts)

        # Find the break point with the maximum counts within the target slope ranges
        try:
            if np.max(break_point_counts[:, 1]) == 0:
                best_break_point = -1
                slope_right = 0
                slope_left = 0
            elif np.max(break_point_counts[:, 1]) == 2:
        #print(max_count_index[:].mean())
        #print(break_point_counts[:,0].mean())
        #print(break_point_counts[max_count_index].mean()
                break_point_counts = break_point_counts[break_point_counts[:, 1] == 2]
                best_break_point = break_point_counts[:,0].mean()
                left_mask =    (x < (best_break_point*gyro)/pad)  & (x > minmin)
                right_mask = (x > (best_break_point*gyro)*pad) & (x<maxmax)
                model_left = LinearRegression().fit(np.log10(x[left_mask]).reshape(-1, 1), np.log10(y[left_mask]))
                model_right = LinearRegression().fit(np.log10(x[right_mask]).reshape(-1, 1), np.log10(y[right_mask]))
                slope_left = model_left.coef_[0]
                slope_right = model_right.coef_[0]


            elif np.max(break_point_counts[:, 1]) == 3:
                break_point_counts = break_point_counts[break_point_counts[:, 1] == 3]

                best_break_point = break_point_counts[:,0].mean()
                left_mask =    (x < (best_break_point)/pad)  & (x > minmin)
                right_mask = (x > (best_break_point)*pad) & (x<maxmax)
                model_left = LinearRegression().fit(np.log10(x[left_mask]).reshape(-1, 1), np.log10(y[left_mask]))
                model_right = LinearRegression().fit(np.log10(x[right_mask]).reshape(-1, 1), np.log10(y[right_mask]))
                slope_left = model_left.coef_[0]
                slope_right = model_right.coef_[0]
                
                if (slope_left < (-5./3.) - thresh) or (slope_left > (-5./3.) + thresh):
                    best_break_point = -1
                    slope_left = 0
                    slope_right = 0
                if (slope_right < (-7./3.) - thresh) or (slope_right > (-7./3.) + thresh):
                    best_break_point = -1
                    slope_left = 0
                    slope_right = 0

                # if slope_left > ((-5/3) + thresh) or slope_left < ((-5/3) - thresh):
                #     slope_left = 0
                #     slope_right = 0
                #     best_break_point = -1
            else:
                best_break_point = -1
                slope_left = 10
                slope_right = 0
        except:
            best_break_point = -1
            slope_left = 0
            slope_right = 0      
        
        if (theoretical_break <= acceptable_freqs_min) or (best_break_point <= 3e-2) :
            #do what needs to be done if the theo break is less than the acceptable min, make all kaw
            left_mask =    0
            right_mask = (x > minmin) & (x<maxmax)
            
            model_left = 0
            model_right = LinearRegression().fit(np.log10(x[right_mask]).reshape(-1, 1), np.log10(y[right_mask]))
            
            slope_left = 0
            slope_right = model_right.coef_[0]
            if best_break_point == -1:
                best_break_point = theoretical_break

            if (slope_right < (-7./3.) - thresh) or (slope_right > (-7./3.) + thresh):
                best_break_point = -1
                slope_left = 0
                slope_right = 0

            

        
        elif (theoretical_break >= acceptable_freqs_max) or (best_break_point >= 7.5e-2) or (best_break_point == -1):
            #do what needs to be done if the theo break is greater than the acceptable max, make all mhd
            left_mask =    (x > minmin) & (x<maxmax)
            right_mask = 0
            
            model_left = LinearRegression().fit(np.log10(x[left_mask]).reshape(-1, 1), np.log10(y[left_mask]))
            model_right = 0
            
            slope_left = model_left.coef_[0]
            slope_right = 0
            
            if best_break_point == -1:
                best_break_point = theoretical_break
            if (slope_left < (-5./3.) - thresh) or (slope_left > (-5./3.) + thresh):
                best_break_point = -1
                slope_left = 0
                slope_right = 0
                
        elif (theoretical_break <= acceptable_freqs_min) and (best_break_point >= acceptable_freqs_max): 
                left_mask =    0
                right_mask = (x > minmin) & (x<maxmax)
                
                model_left = 0
                model_right = LinearRegression().fit(np.log10(x[right_mask]).reshape(-1, 1), np.log10(y[right_mask]))
                
                slope_left = 0
                slope_right = model_right.coef_[0]
                if best_break_point == -1:
                    best_break_point = theoretical_break
                if (slope_right < (-7./3.) - thresh) or (slope_right > (-7./3.) + thresh):
                    best_break_point = -1
                    slope_left = 0
                    slope_right = 0
        
        elif (theoretical_break >= acceptable_freqs_max) and (best_break_point <= acceptable_freqs_min): 
                left_mask =    (x > minmin) & (x<maxmax)
                right_mask = 0
                
                model_left = LinearRegression().fit(np.log10(x[left_mask]).reshape(-1, 1), np.log10(y[left_mask]))
                model_right = 0
                
                slope_left = model_left.coef_[0]
                slope_right = 0
                if best_break_point == -1:
                    best_break_point = theoretical_break
                if (slope_left < (-5./3.) - thresh) or (slope_left > (-5./3.) + thresh):
                    best_break_point = -1
                    slope_left = 0
                    slope_right = 0

        
  

    return best_break_point, slope_left, slope_right, x, y, left_mask, right_mask, keep_index


def find_nearest_index_value(df, target_datetime):
    # Convert target to Timestamp if it's not already
    target = pd.to_datetime(target_datetime)

    # Find the nearest index
    nearest_index = df.index.get_loc(target, method='nearest')

    # Retrieve the nearest index and value
    nearest_datetime = df.index[nearest_index]
    #nearest_value = df[str_param].iloc[nearest_index]
    
    return nearest_datetime#, nearest_value

def mean_changer(df):
    mean_x = df['MEAN_BX']
    mean_y = df['MEAN_BY']
    mean_z = df['MEAN_BZ']
    raw_x = df['BX']
    raw_y = df['BY']
    raw_z = df['BZ']
        
    mean_vecs = df[['MEAN_BX', 'MEAN_BY', 'MEAN_BZ']]
    mean_magnitude = np.sqrt((mean_vecs**2).sum(axis=1))

    
    perturbed_x = raw_x - mean_x
    perturbed_y = raw_y - mean_y
    perturbed_z = raw_z - mean_z
    
    perturbed_df = pd.DataFrame({'B_x_pert': perturbed_x, 'B_y_pert': perturbed_y, 'B_z_pert': perturbed_z})
    perturbed_df_perp = pd.DataFrame({'B_x_pert_perp': [], 'B_y_pert_perp': [], 'B_z_pert_perp': []})
    perturbed_df_par = pd.DataFrame({'B_x_pert_par': [], 'B_y_pert_par': [], 'B_z_pert_par': []})
    par_mag_df = pd.DataFrame({'B_pert_par': []})
                                    
    for i in range(len(perturbed_x)):
        new_df_index = pd.DatetimeIndex([df.index[i].isoformat()])
        mean_vecs = df[['MEAN_BX', 'MEAN_BY', 'MEAN_BZ']]
        parallel_hat = mean_vecs.iloc[i] / mean_magnitude[i]
        perturbed_parallel_mag = np.dot((perturbed_df.iloc[i].to_numpy()), parallel_hat)
        perturbed_parallel_vector = np.asarray(perturbed_parallel_mag*parallel_hat)
        perturbed_par_x = perturbed_parallel_vector[0]
        perturbed_par_y = perturbed_parallel_vector[1]
        perturbed_par_z = perturbed_parallel_vector[2]
        perturbed_perp_x = perturbed_df.iloc[i]['B_x_pert'] - perturbed_par_x
        perturbed_perp_y = perturbed_df.iloc[i]['B_y_pert'] - perturbed_par_y
        perturbed_perp_z = perturbed_df.iloc[i]['B_z_pert'] - perturbed_par_z
        
        
        perturbed_df_perp = perturbed_df_perp.append(
                pd.DataFrame({'B_x_pert_perp': perturbed_perp_x, 'B_y_pert_perp': perturbed_perp_y, 'B_z_pert_perp': perturbed_perp_z},
                             index=new_df_index)
            )
        perturbed_df_par = perturbed_df_par.append(
        pd.DataFrame({'B_x_pert_par': perturbed_par_x, 'B_y_pert_par': perturbed_par_y, 'B_z_pert_par': perturbed_par_z},
                        index=new_df_index)
        )
        par_mag_df = par_mag_df.append(
        pd.DataFrame({'B_pert_par': perturbed_parallel_mag},
                        index=new_df_index)
        )
    data_df = pd.concat([df, perturbed_df_perp], axis=1)
    data_df  = pd.concat([data_df, perturbed_df_par], axis=1)
    data_df = pd.concat([data_df, par_mag_df], axis = 1)
    return data_df

def _freqrange(minf, maxf, f, fb, psd,pad,left_mask,right_mask):
    """Finds ranges of freqencies for MHD and KAW scales.\n  
    Inputs: f is frequency range for PSD, and gyro is the gyrofreqency for given domain. \n
    Returns the two frequency arrays and indices (tuples) for the two arrays. \n b1
    corresponds to MHD, b2 to KAW, and b3 to all real points of freqency range."""

    b1 = left_mask
    freq_mhd = f[b1]
    

    b2 = right_mask
    freq_kaw = f[b2]
    
    try:
        if b1 ==0:
            freq_mhd = None
        pass
    except:
        pass
    try:
        if b2 ==0:
            freq_kaw = None
        pass
    except:
        pass
    
    b3 = np.where((f>0) & (f<0.5)) 

    b4  = np.where((f>minf) & (f<maxf) & (psd>0))  
    all_needed_freqs = f[b4]             

    return freq_mhd, freq_kaw, all_needed_freqs, b1, b2, b3, b4

def _q_calc(one_equation, psd_perp, freq, b1, b2, b4,  v_rel, density,rho_i, theta_rad):
    """Takes PSD of perpendicular component and other parameters to find q MHD 
    and q KAW.  \n Every parameter must be in base units (T, kg, m, etc).  \n Empirical
    parameters are subject to adjustment below.  \n Outputs ranges of q MHD and q KAW 
    over freqency domains, according to b1 and b2 (respectively MHD and KAW freqency 
    domains. \n MAG vector components used only to find theta for k perp.  """
    
    
    
    delta_b_perp3 = (psd_perp*freq)**(3/2)                                                
    
    
    

    mu_0 = np.pi*4*1e-7
    kperp = (2*np.pi*freq)/(v_rel*np.sin(theta_rad))  
    
    sintheta_mean = (np.sin(theta_rad)).mean()


    kperp_mean_mhd = kperp[b1].mean()
    
    v_mean = v_rel.mean()
    
    


    qkaw = (0.5*(delta_b_perp3[b2])*kperp[b2]/np.sqrt(mu_0**3*density))*(1+kperp[b2]**2*rho_i**2)**0.5*(1+(1/(1+kperp[b2]**2*rho_i**2))*(1/(1+1.25*kperp[b2]**2*rho_i**2))**2)
    qmhd = (delta_b_perp3[b1])*kperp[b1]/(np.sqrt((mu_0**3)*density))
    q_mhd_freqs = freq[b1]
    q_all_freqs = freq[b4]
    q_kaw_freqs = freq[b2]
    kperps = kperp[b1]
    rho_i = rho_i
    
    if one_equation:
        qmhd = (0.5*(delta_b_perp3[b1])*kperp[b1]/np.sqrt(mu_0**3*density))*(1+kperp[b1]**2*rho_i**2)**0.5*(1+(1/(1+kperp[b1]**2*rho_i**2))*(1/(1+1.25*kperp[b1]**2*rho_i**2))**2)
        q_all = (0.5*(delta_b_perp3[b4])*kperp[b4]/np.sqrt(mu_0**3*density))*(1+kperp[b4]**2*rho_i**2)**0.5*(1+(1/(1+kperp[b4]**2*rho_i**2))*(1/(1+1.25*kperp[b4]**2*rho_i**2))**2)
    mean_delta_bperp3_psd_mhd = delta_b_perp3[b1].mean()
    
    freqs_mean_mhd = q_mhd_freqs.mean()
    try:
        if b1 == 0:
            qmhd = 100
        pass
    except:
        pass
    try:
        if b2 ==0:
            qkaw = 100
        pass
    except:
        pass
    
    return q_all, qmhd, qkaw, q_mhd_freqs, q_kaw_freqs, q_all_freqs, kperps, rho_i, mean_delta_bperp3_psd_mhd, kperp_mean_mhd, sintheta_mean, v_mean, freqs_mean_mhd


m = 23/6.0229e26
z = 1.6
q = 1.6e-19



#Here is where the actuall q function is ... This version will be hardcoded without the empirical formula to make sure it takes exact densities

def _get_q(case_study, case_directory, compression_df,one_equation, theo_break, z_mag_df,delta_v_df,minf_index, maxf_index, minf, maxf, data_df, part_df, interval, dt, filter_magnitude, fits, moments,orbit_number, case_chance, test,empirical =False, remove_coi = False,acceptable_freqs_min = 2e-2, acceptable_freqs_max = 8e-2):
    if not remove_coi:
        print('not gonna remove cone')
    else:
        print('gonna remove cone')
    #Wherever it says "data_df" was originally "self.data_df". This was changed because the mean field takes a long time to run, it is now a seperate input when the class is called
    
    amu = 1.67*10**-27
    
    m16 = 24*amu
    mo2 = 16*amu
    ms = 32*amu
    ms3 = 32*amu
    
    
    mean_q_mhd = np.array([])
    mean_q_kaw = np.array([])
    
    mean_q = np.array([])
    mean_z = np.array([])
    mean_R = np.array([])
    mean_b_perp = np.array([])
    mean_density_array = np.array([])
    q_mean_whole_range = np.array([])
    mean_vphi_array = np.array([])
    
    
    mean_sintheta_array = np.array([])
    
    mean_freqs_array = np.array([])
    
    mean_v_array = np.array([])
    
    t=1

    mean_kperps = np.array([])
    mean_delta_bperp3_array = np.array([])

    q_kaw_slopes = []
    q_mhd_slopes = []
    breaks = []
    gyros = []
    rho_i_array = []
    theta_array = []
    avg_times = []
    local_time_array = []
    num_intervals = np.ceil((data_df.index.max().to_pydatetime()
                        - data_df.index.min().to_pydatetime())/(timedelta(minutes=interval)))
    for i in range(0,int(num_intervals)):
        start_index = (data_df.index[0]
                        + timedelta(minutes=i*interval))
        end_index = (data_df.index[0]
                        + timedelta(minutes=(i+1)*interval))
        interval_time_series = data_df[start_index: end_index].index
        
        avg_index = pd.to_datetime(start_index) + pd.Timedelta(minutes=interval/2)

        if ((pd.to_datetime(end_index) - pd.to_datetime(start_index)) <  (pd.Timedelta(minutes = interval+5)) ) and ((pd.to_datetime(end_index) - pd.to_datetime(start_index)) > (pd.Timedelta(minutes = interval-5))):
            interval_R = data_df['R'][start_index: end_index].to_numpy()
            
            
            interval_Z = z_mag_df[start_index: end_index].to_numpy()
            mean_R_value = interval_R.mean()
            mean_Z_value = interval_Z.mean()
            #interval_perptotal = data_df['B_PERP_TOT'][start_index: end_index].to_numpy()
            interval_perp_x = data_df['B_x_pert_perp'][start_index: end_index].to_numpy()
            interval_perp_y = data_df['B_y_pert_perp'][start_index: end_index].to_numpy()
            interval_perp_z = data_df['B_z_pert_perp'][start_index: end_index].to_numpy()
            interval_perptotal = np.sqrt((interval_perp_x.mean())**2+(interval_perp_y.mean())**2+(interval_perp_z.mean())**2)
            interval_par_x = data_df['B_x_pert_par'][start_index: end_index].to_numpy()
            interval_par_y = data_df['B_y_pert_par'][start_index: end_index].to_numpy()
            interval_par_z = data_df['B_z_pert_par'][start_index: end_index].to_numpy()
            interval_MEAN_BX = data_df['MEAN_BX'][start_index: end_index].to_numpy()*1e-9
            interval_MEAN_BY = data_df['MEAN_BY'][start_index: end_index].to_numpy()*1e-9
            interval_MEAN_BZ = data_df['MEAN_BZ'][start_index: end_index].to_numpy()*1e-9
            interval_compression_df = compression_df[start_index: end_index]
            interval_compression_df_index = interval_compression_df.index

            interval_Br = data_df['BR'][start_index: end_index].to_numpy()
            interval_Btheta = data_df['BTHETA'][start_index: end_index].to_numpy()
            interval_Bphi = data_df['BPHI'][start_index: end_index].to_numpy()
            try:
                interval_v_perp_x = delta_v_df['u_x_pert_perp'][start_index: end_index].to_numpy()*1e3
                interval_v_perp_y = delta_v_df['u_y_pert_perp'][start_index: end_index].to_numpy()*1e3
                interval_v_perp_z = delta_v_df['u_z_pert_perp'][start_index: end_index].to_numpy()*1e3
            except:
                continue
            interval_v_perp_index = delta_v_df['u_x_pert_perp'][start_index: end_index].index
            interval_B_perp_index = data_df['B_x_pert_perp'][start_index: end_index].index

            mean_Br = interval_Br.mean()
            mean_Btheta =interval_Btheta.mean()
            mean_Bphi = interval_Bphi.mean()

            mean_delta_b_perp_value = interval_perptotal.mean()
            
            



            try:
                len(interval_perp_x)
                if len(interval_perp_x) < 1:
                    #print('too short')
                    continue
            except:
                print('too short')
                continue
            interval_mean_B_mag = np.sqrt(data_df['MEAN_BX'][start_index: end_index]**2+data_df['MEAN_BY'][start_index: end_index]**2 +data_df['MEAN_BZ'][start_index: end_index]**2).to_numpy()
            if not filter_magnitude:
                pass
            else:
                print('filtering')
                if interval_perptotal >= filter_magnitude*interval_mean_B_mag.mean():
                    continue
                else:
                    pass
            
            if empirical:
                pass
                '''
                try:
                    interval_gyro = _gyro(data_df['BX'][start_index: end_index]*10**-9, data_df['BY'][start_index: end_index]*10**-9, data_df['BZ'][start_index: end_index]*10**-9,m , z, q)
                    interval_R = data_df['R'][start_index: end_index].to_numpy()
                    interval_Z = data_df['Z_MAG'][start_index: end_index].to_numpy()
                    vco = (12.6)*interval_R*(1e3)
                    #interval_vphi = vco/((1+(interval_R/50)**2))
                    interval_vphi = vco/((1+(interval_R/50)**2))
                    #interval_vphi = vco*((1.12-(interval_R/50)))
                    a1 = 1987
                    b1 = -8.2
                    a2 = 14
                    b2 = -3.2
                    a3 = 0.05
                    b3 = -0.65
                    interval_n =a1*(interval_R/6)**b1 + a2*(interval_R/6)**b2 +a3*(interval_R/6)**b3
                    mean_n = interval_n.mean()*(100**3)
                    mean_density = (23/6.0229e26)*mean_n
                    mean_temp = _temp_dist(interval_R).mean()
                    mean_vth = _v_therm(mean_temp, m).mean()
                    mean_vphi = interval_vphi.mean()

                   # if interval_R.mean()>20:
                    #    mean_vphi = 200e3
                   # else:
                    #    mean_vphi = interval_vphi.mean()

                    interval_rho_i = _rho_i(mean_vth, interval_gyro)
                    mean_rho_i = interval_rho_i
                except:
                    continue
                '''
            elif fits:
                interval_vphi = part_df['   "ur"'][start_index: end_index].dropna()
                if len(interval_vphi)<1:
                    continue
                else:
                    try:
                        interval_vphi = part_df[' "uphi"'][start_index: end_index].to_numpy()
                        interval_vr = part_df['   "ur"'][start_index: end_index].to_numpy()
                        interval_vtheta = part_df['   "uz"'][start_index: end_index].to_numpy()
                        interval_n = part_df['      "n"'][start_index: end_index].to_numpy()
                        interval_m16 = part_df['r[MQ16]'][start_index: end_index].to_numpy()
                        interval_mo = part_df['r[O2+]'][start_index: end_index].to_numpy()
                        interval_ms = part_df['r[S+]'][start_index: end_index].to_numpy()
                        interval_ms3 = part_df['r[S3+]'][start_index: end_index].to_numpy()
                        interval_temp = part_df['    "T"'][start_index: end_index].to_numpy()
                        interval_local_time = part_df[' "LT"'][start_index: end_index].to_numpy()
                        datetime_list = pd.date_range(start=start_index, end=end_index, freq='1s')
                        datetime_series = pd.Series(datetime_list)
                        mean_datetime = pd.to_datetime(datetime_series.mean())
                        mean_vphi = np.nanmean(interval_vphi)*1e3
                        mean_vr = np.nanmean(interval_vr)*1e3
                        mean_vtheta = np.nanmean(interval_vtheta)*1e3
                        mean_n = np.nanmean(interval_n)
                        mean_m16 = np.nanmean(interval_m16)
                        mean_mo = np.nanmean(interval_mo)
                        mean_ms = np.nanmean(interval_ms)
                        mean_ms3 = np.nanmean(interval_ms3)
                        mean_temp = np.nanmean(interval_temp)
                        mean_m = mean_m16*m16 + mean_mo*mo2 +mean_ms*ms+mean_ms3*ms3
                        interval_gyro = _gyro(data_df['BX'][start_index: end_index]*10**-9, data_df['BY'][start_index: end_index]*10**-9, data_df['BZ'][start_index: end_index]*10**-9,mean_m , z, q)
                        mean_density = mean_m * mean_n*1e6
                        mean_vth = _v_therm(mean_temp, mean_m)
                        mean_rho_i = _rho_i(mean_vth, interval_gyro)
                        mean_local_time = np.nanmean(interval_local_time)
                    except:
                        continue
            elif moments:
                try:
                    interval_vphi = part_df['vphi'][start_index: end_index].to_numpy()
                    interval_vr = part_df['vr'][start_index: end_index].to_numpy()
                    interval_vtheta = part_df['vtheta'][start_index: end_index].to_numpy()
                    interval_n = part_df['n'][start_index: end_index].to_numpy()
                    interval_m = 3.2*10**-26
                    interval_temp = part_df['Temp'][start_index: end_index].to_numpy()
                    interval_local_time = part_df['SC_POS_LOCAL_TIME'][start_index: end_index].to_numpy()
                    datetime_list = pd.date_range(start=start_index, end=end_index, freq='1s')
                    datetime_series = pd.Series(datetime_list)
                    mean_datetime = pd.to_datetime(datetime_series.mean())
                    if len(interval_vphi)<1:
                        mean_vphi = (part_df['vphi'][find_nearest_index_value(part_df, mean_datetime)])*1e3
                        mean_vr = (part_df['vr'][find_nearest_index_value(part_df, mean_datetime)])*1e3
                        mean_vtheta = (part_df['vtheta"'][find_nearest_index_value(part_df, mean_datetime)])*1e3
                        #  mean_R = part_df['  "R"'][find_nearest_index_value(part_df, mean_datetime)]
                        mean_n = part_df['n'][find_nearest_index_value(part_df, mean_datetime)]
                        mean_m = interval_m
                        interval_gyro = _gyro(data_df['BX'][start_index: end_index]*10**-9, data_df['BY'][start_index: end_index]*10**-9, data_df['BZ'][start_index: end_index]*10**-9,mean_m , z, q)
                        mean_density = mean_n* mean_m*1e6
                        #  mean_temp = part_df['    "T"'][find_nearest_index_value(part_df, mean_datetime)]
                        mean_temp = _temp_dist(mean_R_value)
                        mean_vth = _v_therm(mean_temp, mean_m)
                        mean_rho_i = _rho_i(mean_vth, interval_gyro)
                        mean_local_time = part_df['SC_POS_LOCAL_TIME'][find_nearest_index_value(part_df, mean_datetime)]
                    else:
                        mean_vphi = interval_vphi.mean()*1e3
                        mean_vr = interval_vr.mean()*1e3
                        mean_vtheta = interval_vtheta.mean()*1e3
                        mean_n = interval_n.mean()
                        mean_m = interval_m
                        mean_temp = interval_temp.mean()
                        interval_gyro = _gyro(data_df['BX'][start_index: end_index]*10**-9, data_df['BY'][start_index: end_index]*10**-9, data_df['BZ'][start_index: end_index]*10**-9,mean_m , z, q)
                        mean_density = mean_m * mean_n*1e6
                        #mean_temp = part_df['    "T"'][find_nearest_index_value(part_df, mean_datetime)]
                        mean_temp = _temp_dist(mean_R_value)
                        mean_vth = _v_therm(mean_temp, mean_m)
                        mean_rho_i = _rho_i(mean_vth, interval_gyro)
                        mean_local_time = interval_local_time.mean()
                except:
                    continue





            fs = int(1/dt)
            dot_prod = mean_vr*mean_Br+mean_vtheta*mean_Btheta+mean_vphi*mean_Bphi
            v_mag = np.sqrt(mean_vr**2+mean_vtheta**2+mean_vphi**2)
            B_mag = np.sqrt(mean_Br**2+mean_Bphi**2+mean_Btheta**2)
            mag_values_mult = v_mag*B_mag
            theta_rad = math.acos(dot_prod/mag_values_mult)
            theta_deg = math.degrees(theta_rad)
            mu_0 = np.pi*4e-7
            
            v_Alfven = ((interval_mean_B_mag*1e-9))/np.sqrt(mu_0*mean_density)
            
            
            
        
            

            try:

                cwt_perp_x = CWTData(interval_time_series,
                                    interval_perp_x,
                                    dt)
            
                cwt_perp_y = CWTData(interval_time_series,
                                    interval_perp_y,
                                    dt)
                
                cwt_perp_z = CWTData(interval_time_series,
                                    interval_perp_z,
                                    dt)
                if remove_coi:
                    cwt_perp_x.remove_coi()
                    cwt_perp_y.remove_coi()
                    cwt_perp_z.remove_coi()
                psd_x = _psd(interval_perp_x, cwt_perp_x.power, cwt_perp_x.freqs, fs)*10**-18
                psd_y = _psd(interval_perp_y, cwt_perp_y.power, cwt_perp_y.freqs, fs)*10**-18
                psd_z = _psd(interval_perp_z, cwt_perp_z.power, cwt_perp_z.freqs, fs)*10**-18
                psd_perp = psd_x + psd_y + psd_z
                #psd_perp = np.sqrt(psd_x**2+psd_y**2+psd_z**2)

                
                psd_perp = psd_perp[np.where(psd_perp>0)]
                freqs = cwt_perp_x.freqs[np.where(psd_perp>0)]
                                            
                K_b = 1.38e-23
                charge = 1.6e-19
                


                rho_i = np.sqrt(2*mean_temp*11606*K_b*mean_m)/(1.6*charge*np.abs(interval_mean_B_mag.mean())*10**-9)
                theta_rad = math.acos(dot_prod/mag_values_mult)
                mean_sintheta = np.sin(np.mean(theta_rad))
                theoretical_break_raw= v_mag*mean_sintheta/(2*np.pi*rho_i)
                

                
                thresh = .33
                

                acceptable_freqs_min = 2e-2    
                break_coeff, mhd_slope, kaw_slope, x, y, left_mask, right_mask, keep_index = get_break_coeff_and_slopes(theo_break,minf_index, maxf_index, freqs, psd_perp, interval_gyro, 1, thresh, theoretical_break_raw, acceptable_freqs_min, acceptable_freqs_max)
            
            except:
                break_coeff = -1
                continue

            if (break_coeff == -1) or (keep_index == 1):
                rho_i = np.sqrt(2*mean_temp*11606*K_b*mean_m)/(1.6*charge*np.abs(interval_mean_B_mag.mean())*10**-9)
                theoretical_break_raw= v_mag*mean_sintheta/(2*np.pi*rho_i)
                mean_q_mhd = np.append(mean_q_mhd, -1)
                mean_q_kaw = np.append(mean_q_kaw, -1)
                mean_q = np.append(mean_q, -1)
                mean_z = np.append(mean_z, interval_Z.mean())
                mean_R = np.append(mean_R, interval_R.mean())
                
                mean_sintheta_array = np.append(mean_sintheta_array, -1)
                
                mean_freqs_array = np.append(mean_freqs_array, -1)
                
                mean_v_array = np.append(mean_v_array, -1)
                
                mean_kperps = np.append(mean_kperps, -1)
                mean_delta_bperp3_array = np.append(mean_delta_bperp3_array, -1)


                local_time_array = np.append(local_time_array, mean_local_time)
                mean_b_perp = np.append(mean_b_perp, mean_delta_b_perp_value)
                mean_density_array = np.append(mean_density_array, mean_density)
                mean_vphi_array = np.append(mean_vphi_array, mean_vphi)
                q_kaw_slopes.append(kaw_slope)
                q_mhd_slopes.append(mhd_slope)
                avg_times.append(avg_index)
                q_mean_whole_range = np.append(q_mean_whole_range,-1)
                continue
            else:
                fb = break_coeff
                freq_mhd, freq_kaw, all_freqs, b1, b2, b3,b4 = _freqrange(minf, maxf,freqs, fb, psd_perp,1,left_mask,right_mask)
                q_all , q_mhd, q_kaw, q_mhd_freqs, q_kaw_freqs,q_all_freqs, kperps, rho_i, delta_bperp3_value_mhd, kperp_value_mhd, mean_sintheta, mean_v, mean_freq = _q_calc(one_equation,psd_perp, freqs,
                                    b1, b2,b4, v_mag, mean_density, rho_i, theta_rad)    #this second b1 was actually b2, just b2 for now to do calc with both equation in same freq range
                mean_sintheta_array = np.append(mean_sintheta_array, mean_sintheta)
                
                q_kaw_slopes.append(kaw_slope)
                q_mhd_slopes.append(mhd_slope)
                breaks.append(fb)
                gyros.append(interval_gyro)
                rho_i_array.append(mean_rho_i)
                q_mean_whole_range = np.append(q_mean_whole_range, np.mean(q_all))
                
                

                if type(q_mhd) is int:
                    mean_q = np.append(mean_q, np.mean(q_kaw))
                elif type(q_kaw) is int:
                    mean_q = np.append(mean_q, np.mean(q_mhd))
                else:
                    mean_q = np.append(mean_q, (np.mean(q_mhd) + np.mean(q_kaw))/2)
                    
                
                
                
                if type(left_mask) is int:
                    mean_q_mhd = np.append(mean_q_mhd, 100)
                else: 
                    mean_q_mhd = np.append(mean_q_mhd, np.mean(q_mhd))

                if type(right_mask) is int:
                    mean_q_kaw = np.append(mean_q_kaw, 100)
                else:
                    mean_q_kaw = np.append(mean_q_kaw, np.mean(q_kaw))
                
                
                

                mean_delta_bperp3_array = np.append(mean_delta_bperp3_array, delta_bperp3_value_mhd)
                mean_kperps = np.append(mean_kperps, kperp_value_mhd)
                
                mean_freqs_array = np.append(mean_freqs_array, mean_freq)
                
                mean_v_array = np.append(mean_v_array, mean_v)



                mean_R = np.append(mean_R, mean_R_value)
                mean_z = np.append(mean_z, mean_Z_value)
                mean_density_array = np.append(mean_density_array, mean_density)
                mean_vphi_array = np.append(mean_vphi_array, mean_vphi)
                theta_array = np.append(theta_array, theta_deg)
                mean_b_perp = np.append(mean_b_perp, mean_delta_b_perp_value)
                avg_times.append(avg_index)
                local_time_array = np.append(local_time_array, mean_local_time)
                
                
                #case study stuff
                
                if case_study:

                    if (mean_R_value < 35) & (mean_R_value > 15) & (np.abs(mean_Z_value) < 2) & (break_coeff !=-1) & (mean_sintheta > 0.1):
                        #print('made it to case study')
                        
                        chance = random.randint(1,100)
                        if chance <= case_chance:
                            

                            directory = case_directory+'/Orbit_'+str(orbit_number) +'_Case_' +str(t)+'/'
                            #if test:
                            #    directory = '/home/mtfranciscovich/Juno-codes/test2_case_study_images_fits/Orbit_'+str(orbit_number) +'_Case_' +str(t)+'_test/'

                            if not os.path.exists(directory):
                                os.makedirs(directory)
                                
                                
                            fig, ax = plt.subplots(6, figsize = (25,12), sharex = 'all')


                            b_perp_tot = np.sqrt(interval_perp_x**2 + interval_perp_y**2 + interval_perp_z**2)
                            b_par_tot = np.sqrt(interval_par_x**2 + interval_par_y**2 + interval_par_z**2)
                            #ax[1].plot(vitaly_data_mean['B_PERP_TOT'], ls = "-.",c='red')
                            
                            
                            ax[0].plot(data_df['R'][start_index: end_index].index, interval_Br)
                            ax[1].plot(data_df['R'][start_index: end_index].index, interval_Btheta)
                            ax[2].plot(data_df['R'][start_index: end_index].index, interval_Bphi)
                            ax[3].plot(data_df['R'][start_index: end_index].index, b_par_tot)
                            ax[4].plot(data_df['R'][start_index: end_index].index, b_perp_tot)
                            ax[5].plot(data_df['R'][start_index: end_index].index, interval_Z)
                            
                            
                            ax[0].set_ylabel(r'$B_r$ [nT]')
                            ax[1].set_ylabel(r'$B_{\theta}$ [nT]')
                            ax[2].set_ylabel(r'$B_{\phi}$ [nT]')
                            ax[3].set_ylabel(r'${\delta}B_{||}$ [nT]')
                            ax[4].set_ylabel(r'${\delta}B_{\perp}$ [nT]')
                            ax[5].set_ylabel(r'$Z_{mag} [R_J]$')
                            
                            
                            ax[0].set_title('Magnetometer Data, L = ' +str(round(mean_R_value,3)) +', $Z_{mag} = $' +str(round(mean_Z_value,3)))

                            
                            plt.savefig(directory+'/magnetometer', facecolor='white')

                            
                            plt.close()
                            
                            fig,ax = plt.subplots(3,figsize =(16,16))
                            cwt_perp_x.cwt_plot(ax[0], remove_coi=False)
                            cwt_perp_y.cwt_plot(ax[1],remove_coi=False)
                            cwt_perp_z.cwt_plot(ax[2],remove_coi=False)


                            ax[0].set_title('Power X Perp, L = ' +str(round(mean_R_value,3)) +', $Z_{mag} = $' +str(round(mean_Z_value,3)))
                            ax[1].set_title('Power Y Perp, L = ' +str(round(mean_R_value,3)) +', $Z_{mag} = $' +str(round(mean_Z_value,3)))
                            ax[2].set_title('Power Z Perp, L = ' +str(round(mean_R_value,3)) +', $Z_{mag} = $' +str(round(mean_Z_value,3)))
                            
                            
                            plt.savefig(directory+'/CWT', facecolor='white')
                            
                            plt.close()
                            plt.figure(figsize=(10, 6))
                            plt.loglog(x, y, color = 'black')
                            
                            if mhd_slope != 0 and kaw_slope != 0:
                                model_left = LinearRegression().fit(np.log10(x[left_mask]).reshape(-1, 1), np.log10(y[left_mask]))
                                model_right = LinearRegression().fit(np.log10(x[right_mask]).reshape(-1, 1), np.log10(y[right_mask]))
                                
                                r2_mhd = r2_score(np.log10(y[left_mask]),model_left.predict(np.log10(x[left_mask]).reshape(-1, 1)))
                                r2_kaw = r2_score(np.log10(y[right_mask]),model_right.predict(np.log10(x[right_mask]).reshape(-1, 1)))
                                
                                plt.loglog(10**np.log10(x[left_mask]), 10**model_left.predict(np.log10(x[left_mask]).reshape(-1, 1)), color='blue', label = 'Slope MHD: ' + str(round(mhd_slope,3))+r', $R^2=$' +str(round(r2_mhd,3)))#, alpha = 0.01)  # Fitted left line
                                plt.loglog(10**np.log10(x[right_mask]), 10**model_right.predict(np.log10(x[right_mask]).reshape(-1, 1)), color='orange', label ='Slope KAW: ' + str(round(kaw_slope,3))+r', $R^2=$' +str(round(r2_kaw,3)))#, alpha=0.01)  # Fitted right line
                                #plt.axvline(x = fb, label='Calculated break')
                                plt.axvline(x = theoretical_break_raw, label = r'$k_{\perp}\rho_i=1$', color = 'green')
                            elif mhd_slope == 0:
                                model_right = LinearRegression().fit(np.log10(x[right_mask]).reshape(-1, 1), np.log10(y[right_mask]))
                                r2_kaw = r2_score(np.log10(y[right_mask]),model_right.predict(np.log10(x[right_mask]).reshape(-1, 1)))
                                plt.loglog(10**np.log10(x[right_mask]), 10**model_right.predict(np.log10(x[right_mask]).reshape(-1, 1)), color='orange', label ='Slope KAW: ' + str(round(kaw_slope,3))+r', $R^2=$' +str(round(r2_kaw,3)))#, alpha=0.01)  # Fitted right line
                               # plt.axvline(x = fb, label='Calculated break')
                                plt.axvline(x = theoretical_break_raw, label = r'$k_{\perp}\rho_i=1$', color = 'green')
                            elif kaw_slope == 0:

                                model_left = LinearRegression().fit(np.log10(x[left_mask]).reshape(-1, 1), np.log10(y[left_mask]))
                                r2_mhd = r2_score(np.log10(y[left_mask]),model_left.predict(np.log10(x[left_mask]).reshape(-1, 1)))

                                plt.loglog(10**np.log10(x[left_mask]), 10**model_left.predict(np.log10(x[left_mask]).reshape(-1, 1)), color='orange', label ='Slope MHD: ' + str(round(mhd_slope,3))+r', $R^2=$' +str(round(r2_mhd,3)))#, alpha=0.01)  # Fitted right line
                                #plt.axvline(x = fb, label='Calculated break')
                                plt.axvline(x = theoretical_break_raw, label = r'$k_{\perp}\rho_i=1$', color = 'green')
                            plt.xlabel('f [Hz]')
                            plt.ylabel('PSD [T^2/Hz]')
                            plt.title('PSD, L = ' +str(round(mean_R_value,3)) +', $Z_{mag} = $' +str(round(mean_Z_value,2)) +', |v| = ' +str(round(mean_v)) + ' m/s, sin(theta_vb) = ' +str(round(mean_sintheta,2))+', Temp. = ' +str(round(mean_temp*11606))+' K')
                            plt.minorticks_on()
                            plt.grid(True, which='minor', axis='both', linestyle=':', color='gray', alpha=0.75)
                            plt.grid(True)
                            plt.legend()
                            plt.savefig(directory+'/PSD', facecolor = 'white')
                            
                            plt.close()
                            plt.figure(figsize=(10, 6))

                            try:
                                
                                if q_mhd == 100:
                                    plt.loglog(q_kaw_freqs, q_kaw)
                                    plt.axhline(y= np.mean(q_kaw), c='red',label = r'$q_{KAW} = $' + str(round(np.mean(q_kaw), 20)))
                            except:
                                pass
                            try:
                                
                                if q_kaw == 100:
                                    plt.loglog(q_mhd_freqs, q_mhd)
                                    plt.axhline(y= np.mean(q_mhd), c='green',label = r'$q_{MHD} = $' + str(round(np.mean(q_mhd), 20)))
                            except:
                                pass
                            try:
                                len_q_mhd = len(q_mhd)
                                len_q_kaw = len(q_kaw)
                                
                                plt.loglog(q_all_freqs, q_all)
                                plt.axhline(y= np.mean(q_all), c='purple',label = r'$q = $' + str(round(np.mean(q_all), 20)))
                                
                                
                                plt.loglog(q_kaw_freqs, q_kaw)
                                plt.axhline(y= np.mean(q_kaw), c='red',label = r'$q_{KAW} = $' + str(round(np.mean(q_kaw), 20)))
                                
                                plt.loglog(q_mhd_freqs, q_mhd)
                                plt.axhline(y= np.mean(q_mhd), c='green',label = r'$q_{MHD} = $' + str(round(np.mean(q_mhd), 20)))
                            except:
                                pass

                            plt.xlabel('f [Hz]')
                            plt.ylabel(r'$q [W/m^3]$')
                            
                            plt.title('Heating Rate Density, L = ' +str(round(mean_R_value,3)) +', $Z_{mag} = $' +str(round(mean_Z_value,2)) +' , |v| = '+str(round(mean_v)) + ' m/s, sin(theta_vb) = ' +str(round(mean_sintheta,2))+', Temp. = ' +str(round(mean_temp*11606))+' K')
                            plt.minorticks_on()
                            plt.grid(True, which='minor', axis='both', linestyle=':', color='gray', alpha=0.75)
                            plt.grid(True)
                            plt.legend()
                            plt.savefig(directory+ '/q_plot', facecolor = 'white')
                            plt.close()
                            
                            
                            fig,ax = plt.subplots(4, figsize=(12, 9), sharex='all')
                            delta_b_perp_mag = np.sqrt(interval_perp_x**2 + interval_perp_y**2 + interval_perp_z**2)
                            norm_b_perp = delta_b_perp_mag/interval_mean_B_mag
                            
                            delta_v_perp_mag = np.sqrt(interval_v_perp_x**2+interval_v_perp_y**2+interval_v_perp_z**2)
                            try:
                                norm_v_perp = delta_v_perp_mag / v_Alfven
                            except: 
                                print('mistmatch')
                                t+=1
                                plt.close()
                                continue

                            
                            ax[0].scatter(interval_v_perp_index, norm_v_perp, label = 'normalized delta_v_perp')
                            ax[0].scatter(interval_B_perp_index, norm_b_perp, label = 'normalized delta_b_perp')
                            
                            ax[1].scatter(interval_v_perp_index, interval_v_perp_x/v_Alfven)
                            ax[1].scatter(interval_B_perp_index, interval_perp_x/interval_mean_B_mag) 
                                
                            ax[2].scatter(interval_v_perp_index, interval_v_perp_y/v_Alfven)
                            ax[2].scatter(interval_B_perp_index, interval_perp_y/interval_mean_B_mag)
                            
                            ax[3].scatter(interval_v_perp_index, interval_v_perp_z/v_Alfven)
                            ax[3].scatter(interval_B_perp_index, interval_perp_z/interval_mean_B_mag)
                            
                                
                            plt.xlabel('Time')
                            #plt.ylabel(r'$q_{MHD} [W/m^3]$')
                            ax[0].set_title('Delta_B_perp vs Delta_v_perp, L = ' +str(round(mean_R_value,3)) +', $Z_{mag} = $' +str(round(mean_Z_value,2)) +', |v| = ' +str(round(mean_v)) + ' m/s, sin(theta_vb) = ' +str(round(mean_sintheta,2))+', Temp. = ' +str(round(mean_temp*11606))+' K')
                            #plt.minorticks_on()
                            #plt.grid(True, which='minor', axis='both', linestyle=':', color='gray', alpha=0.75)
                            #plt.grid(True)
                            ax[0].legend()
                            ax[0].set_ylabel('Magnitude')
                            ax[1].set_ylabel('Perp X')
                            ax[2].set_ylabel('Perp Y')
                            ax[3].set_ylabel('Perp Z')

                            plt.savefig(directory+'/variations', facecolor = 'white')
                            plt.close()
                            
                            fig,ax = plt.subplots(4, figsize=(12, 9), sharex='all')
                            
                            window_size=1
                            mean_perp_data = pd.DataFrame({'MEAN_BX_perp': [],
                                    'MEAN_BY_perp': [],
                                    'MEAN_BZ_perp' : [],
                                    'MEAN_B_perp_mag' : [],
                                    'MEAN_B_mean': []
})
                            try:
                                interval_v_perp_index_no_na = delta_v_df['u_x_pert_perp'][start_index: end_index].dropna().index


                                finish_datetime = (interval_v_perp_index_no_na[-1]
                            - timedelta(minutes=np.floor(window_size/2)))
                                for datetime_index in interval_v_perp_index_no_na:
                                    start_datetime = datetime_index - timedelta(seconds = 30)
                                    end_datetime = start_datetime + timedelta(seconds = 60)
                                    mean_datetime_index = pd.DatetimeIndex([
                                    (start_datetime + timedelta(seconds=30)).isoformat()
                                    ])
                                    temp_mean_Bx_perp = np.nanmean(interval_perp_x[np.where( (interval_B_perp_index >= start_datetime.isoformat()) & (interval_B_perp_index <= end_datetime.isoformat()) )] )
                                    temp_mean_By_perp = np.nanmean(interval_perp_y[np.where( (interval_B_perp_index >= start_datetime.isoformat()) & (interval_B_perp_index <= end_datetime.isoformat()) )] )
                                    temp_mean_Bz_perp = np.nanmean(interval_perp_z[np.where( (interval_B_perp_index >= start_datetime.isoformat()) & (interval_B_perp_index <= end_datetime.isoformat()) )] )
                                    temp_mean_B_perp_mag = np.sqrt(temp_mean_Bx_perp**2+temp_mean_By_perp**2+temp_mean_Bz_perp**2)
                                    temp_mean_B_Mean = np.nanmean(interval_mean_B_mag[np.where( (interval_B_perp_index >= start_datetime.isoformat()) & (interval_B_perp_index <= end_datetime.isoformat()) )] )

                                    mean_array_to_concat = pd.DataFrame(
                                    {'MEAN_BX_perp': temp_mean_Bx_perp,
                                        'MEAN_BY_perp': temp_mean_By_perp,
                                        'MEAN_BZ_perp' : temp_mean_Bz_perp,
                                        'MEAN_B_perp_mag' : temp_mean_B_perp_mag,
                                        'MEAN_B_mean': temp_mean_B_Mean
                                        
                                        }, index=mean_datetime_index)
                                    mean_perp_data = pd.concat([mean_perp_data, mean_array_to_concat])
                                
                                    #if mean_datetime_index == finish_datetime:
                                    #   break
                                #mean_perp_df = mean_perp_df[mean_energy_data.index[0].isoformat(): 
                                #                        mean_energy_data.index[-1].isoformat()]
                                #mean_perp_df = pd.concat([energy_df, mean_energy_data], axis=1)

                                
                                
                                
                                #delta_b_perp_mag = np.sqrt(interval_perp_x**2 + interval_perp_y**2 + interval_perp_z**2)
                                #norm_b_perp = delta_b_perp_mag/interval_mean_B_mag
                                
                                delta_v_perp_mag = np.sqrt(interval_v_perp_x**2+interval_v_perp_y**2+interval_v_perp_z**2)
                                
                                
                                norm_v_perp = delta_v_perp_mag / v_Alfven
                                
                                    
                                
                                
                                ax[0].scatter(interval_v_perp_index, norm_v_perp, label = 'normalized delta_v_perp')
                                ax[0].scatter(mean_perp_data.index, mean_perp_data['MEAN_B_perp_mag']/mean_perp_data['MEAN_B_mean'], label = 'normalized delta_b_perp',marker = '*')
                                
                                ax[1].scatter(interval_v_perp_index, interval_v_perp_x/v_Alfven)
                                ax[1].scatter(mean_perp_data.index, mean_perp_data['MEAN_BX_perp']/mean_perp_data['MEAN_B_mean'],marker = '*') 
                                    
                                ax[2].scatter(interval_v_perp_index, interval_v_perp_y/v_Alfven)
                                ax[2].scatter(mean_perp_data.index, mean_perp_data['MEAN_BY_perp']/mean_perp_data['MEAN_B_mean'],marker = '*')
                                
                                ax[3].scatter(interval_v_perp_index, interval_v_perp_z/v_Alfven)
                                ax[3].scatter(mean_perp_data.index, mean_perp_data['MEAN_BZ_perp']/mean_perp_data['MEAN_B_mean'],marker = '*')
                                
                                    
                                plt.xlabel('Time')
                                ax[0].set_ylabel('Magnitude')
                                ax[1].set_ylabel('Perp X')
                                ax[2].set_ylabel('Perp Y')
                                ax[3].set_ylabel('Perp Z')

                                #plt.ylabel(r'$q_{MHD} [W/m^3]$')
                                ax[0].set_title('Delta_B_perp vs Delta_v_perp, L = ' +str(round(mean_R_value,3)) +', $Z_{mag} = $' +str(round(mean_Z_value,2)) +', |v| = ' +str(round(mean_v)) + ' m/s, sin(theta_vb) = ' +str(round(mean_sintheta,2))+', Temp. = ' +str(round(mean_temp*11606))+' K')
                                #plt.minorticks_on()
                                #plt.grid(True, which='minor', axis='both', linestyle=':', color='gray', alpha=0.75)
                                #plt.grid(True)
                                ax[0].legend()
                                plt.savefig(directory+'/variations2', facecolor = 'white')
                                plt.close()
                                
                                
                                
                                sorted_dfs_desc_R = interval_compression_df
                                
                                fig, ax = plt.subplots(4, figsize = (12,8),sharex='all')
                                
                                

                                size = 20

                                delta_rho_over_delta_v_perp = np.abs(sorted_dfs_desc_R['delta_density'])/sorted_dfs_desc_R['delta_v_perp_mag']
                                v_p = np.sqrt(sorted_dfs_desc_R['v_Alfven']**2 + sorted_dfs_desc_R['v_S']**2)

                                rho_over_vp = sorted_dfs_desc_R['MEAN_density']/v_p
                                
                                deltas_ratio = delta_rho_over_delta_v_perp/rho_over_vp

                                delta_v_perp_over_v_p = sorted_dfs_desc_R['delta_v_perp_mag']/v_p

                                delta_b_perp_over_b = sorted_dfs_desc_R['B_pert_perp_mag']/sorted_dfs_desc_R['MEAN_B']

                                delta_b_par_over_b = sorted_dfs_desc_R['B_pert_par_mag']/sorted_dfs_desc_R['MEAN_B']


                                ax[0].scatter(interval_compression_df_index,deltas_ratio,s=size)
                                ax[0].axhline(y=1,color='red')

                                ax[0].set_title(r'$\frac{|\delta \rho|}{|\delta v_{\perp}|}/\frac{\rho}{v_p}$')
                                
                                
                                

                                ax[2].scatter(interval_compression_df_index, sorted_dfs_desc_R['v_S']/sorted_dfs_desc_R['v_Alfven'],s=size)
                                ax[2].axhline(y=1,color='red')
                                
                                
                                ax[1].scatter(interval_compression_df_index, sorted_dfs_desc_R['v_S'],s=size)
                                
                                
                                ax[1].set_title(r'$v_S$')
                                ax[2].set_title(r'$v_S/v_{A}$')

                                ax[3].scatter(interval_compression_df_index, delta_b_perp_over_b, label = r'$\delta B_{\perp}/B$',s=size,alpha=0.25 )
                                ax[3].scatter(interval_compression_df_index, delta_b_par_over_b, label = r'$\delta B_{||}/B$',s=size,alpha=0.25)
                                ax[3].scatter(interval_compression_df_index, delta_v_perp_over_v_p, c='k',label = r'$\delta v_{\perp}/v_p$',s=size*10, marker = 'x')
                                

                                ax[0].set_yscale('log')
                                ax[1].set_yscale('log')
                                ax[2].set_yscale('log')
                                ax[3].set_yscale('log')


                                ax[3].legend()
                                
                                

                                plt.savefig(directory+'/compression', facecolor = 'white')

                                plt.close()
                                '''
                                sorted_dfs_desc_R = interval_compression_df_2
                                interval_compression_df_index = interval_compression_df_2.index
                                
                                fig, ax = plt.subplots(4, figsize = (12,8),sharex='all')
                                
                                
                                
                                
                                size = 20
                                
                                print('here')

                                delta_rho_over_delta_v_perp = np.abs(sorted_dfs_desc_R['delta_density'])/sorted_dfs_desc_R['delta_v_perp_mag']
                                v_p = np.sqrt(sorted_dfs_desc_R['v_Alfven']**2 + sorted_dfs_desc_R['v_S']**2)

                                rho_over_vp = sorted_dfs_desc_R['MEAN_density']/v_p
                                
                                deltas_ratio = delta_rho_over_delta_v_perp/rho_over_vp

                                delta_v_perp_over_v_p = sorted_dfs_desc_R['delta_v_perp_mag']/v_p

                                delta_b_perp_over_b = sorted_dfs_desc_R['B_pert_perp_mag']/sorted_dfs_desc_R['MEAN_B']

                                delta_b_par_over_b = sorted_dfs_desc_R['B_pert_par_mag']/sorted_dfs_desc_R['MEAN_B']


                                ax[0].scatter(interval_compression_df_index,deltas_ratio,s=size)
                                ax[0].set_title(r'$\frac{|\delta \rho|}{|\delta v_{\perp}|}/\frac{\rho}{v_p}$')
                                
                                
                                

                                ax[2].scatter(interval_compression_df_index, sorted_dfs_desc_R['v_S']/sorted_dfs_desc_R['v_Alfven'],s=size)
                                
                                
                                ax[1].scatter(interval_compression_df_index, sorted_dfs_desc_R['v_S'],s=size)
                                
                                
                                ax[1].set_title(r'$v_S$')
                                ax[2].set_title(r'$v_S/v_{A}$')

                                ax[3].scatter(interval_compression_df_index, delta_b_perp_over_b, label = r'$\delta B_{\perp}/B$',s=size,alpha=0.25 )
                                ax[3].scatter(interval_compression_df_index, delta_b_par_over_b, label = r'$\delta B_{||}/B$',s=size,alpha=0.25)
                                ax[3].scatter(interval_compression_df_index, delta_v_perp_over_v_p, c='k',label = r'$\delta v_{\perp}/v_p$',s=size*10, marker = 'x')
                                

                                ax[0].set_yscale('log')
                                ax[1].set_yscale('log')
                                ax[2].set_yscale('log')
                                ax[3].set_yscale('log')


                                ax[3].legend()
                                
                                

                                plt.savefig(directory+'/compression2', facecolor = 'white')

                                plt.close()
                                '''

                            except:
                                plt.close()
                                t+=1
                                pass
                                
                            
                            
                            t+=1

        else:
            continue
        
    time = pd.date_range(data_df.index[0].isoformat(),
                        data_df.index[-1].isoformat(),
                        periods=len(mean_q))

          
    q_data = pd.DataFrame({'q': mean_q,
                            'q_kaw': mean_q_kaw,
                            'q_mhd': mean_q_mhd,
                            'mean_R': mean_R,
                            'mean_Z': mean_z,
                            'mean_density': mean_density_array,
                            'mean_vphi': mean_vphi_array,
                            'mean_b_perp' : mean_b_perp,
                            'avg_times' : avg_times,
                            'local_times': local_time_array,
                            'kperps': mean_kperps,
                            'mean_deltabperp3': mean_delta_bperp3_array,
                            'sintheta': mean_sintheta_array,
                            '|v|': mean_v_array,
                            'freq_mhd': mean_freqs_array,
                            'q_mean_whole_range': q_mean_whole_range
                            
                            },index=time)

    plt.cla()
    return q_data, q_mhd_slopes, q_kaw_slopes, breaks, gyros, rho_i_array, theta_array



Reading in fits data...

In [ ]:
fits_data = pd.read_csv('/data/juno_spacecraft/data/FWD_FITS/JADE-I_forward_model_ion_v1/JADE-I_forward_model_ion_all_v1.csv', delimiter= ',')
def convert_datetime(datetime_str):
    # Remove the microseconds part
    datetime_str_cleaned = datetime_str.split('.')[0]
    # Convert to pandas datetime object
    date_time = pd.to_datetime(datetime_str_cleaned, format='%Y-%jT%H:%M:%S')
    # Format to the desired output
    return pd.to_datetime(date_time.strftime('%Y-%m-%dT%H:%M:%S'))

fits_data['DATETIME'] = fits_data['                "UTC"'].apply(convert_datetime)
fits_data = fits_data.drop(columns='                "UTC"')

fits_data = fits_data.set_index('DATETIME', drop=True)




In [ ]:
saving_pickles = True
time_res = '1s'
mean_window = '4min'
interval = 20 
startOrbitNumber = 6
endOrbitNumber =  54


name_of_dir='slope_limits_20min_jsscorrected_psdcorrected'

#mean_mag_data_files_dir = '/home/mtfranciscovich/Juno-codes/jss_reduced_inbound_1s_means_4min_pickles'  #This is the filename for each orbit's magdata means in JSS coordinates (Also R data)
mean_mag_data_files_dir = '/home/mtfranciscovich/Juno-codes/means_4min_reduced_inbound_1s_pickles_unit_z_mag'
mag_data_mag_frame_dir = '/home/mtfranciscovich/Juno-codes/reduced_inbound_1s_pickles' #This is the magdata in the magframe (Z_MAG data)
v_perps_dir = '/home/mtfranciscovich/Juno-codes/test_fits_u_cart_means_4min' #This is the v_perp data (only necessary in case study)


directory_to_store_q_pickles_parent = '/home/mtfranciscovich/Juno-codes/'+name_of_dir+'/'+str(interval)+'min_qs_fits_jss_keepcone_nointerp_pickles'
directory_to_store_orbit_qs = directory_to_store_q_pickles_parent+'/orbit_qs'
if not os.path.exists(directory_to_store_orbit_qs):
    os.makedirs(directory_to_store_orbit_qs)
    
directory_to_store_q_pickles_extras_indexed = directory_to_store_q_pickles_parent+'/orbit_indexed'
if not os.path.exists(directory_to_store_q_pickles_extras_indexed):
    os.makedirs(directory_to_store_q_pickles_extras_indexed)
    
directory_to_store_q_pickles_extras_not_indexed = directory_to_store_q_pickles_parent+'/orbit_not_indexed'
if not os.path.exists(directory_to_store_q_pickles_extras_not_indexed):
    os.makedirs(directory_to_store_q_pickles_extras_not_indexed)


do_case_study = True
case_study_parent = '/home/mtfranciscovich/Juno-codes/'+name_of_dir+'/test2_case_study_images_fits'
if do_case_study:
    if not os.path.exists(case_study_parent):
        os.makedirs(case_study_parent)








for i in range(startOrbitNumber, endOrbitNumber+1):
    case_study_dir = case_study_parent

    print('Grabbing Orbit #' +str(i)+' Mag Data...')
    #filename = mean_mag_data_files_dir+'/reduced_orbit_' +str(i)+'_'+str(mean_window)+ '_means_'  +str(time_res) +'.pkl' 
    filename = mean_mag_data_files_dir+'/reduced_orbit_' + str(i) +'_1s.pkl'
    picklefile = open(filename,'rb')
    orbit_means = pickle.load(picklefile)
    #print(orbit_means.columns)
    #continue
    filename = mag_data_mag_frame_dir+'/reduced_orbit_' +str(i) +'_1s.pkl' #This is the magdata in the magframe (Z_MAG data)
    picklefile = open(filename,'rb')
    mag_data_mag_frame = pickle.load(picklefile)
    z_mag_info = mag_data_mag_frame['Z_MAG'] #The Z_mag data from the mag frame data file
    R_info = mag_data_mag_frame['R']
    
    
    
    print('Grabbing Orbit #' +str(i)+' v_perps...')
    filename = v_perps_dir+'/u_cart_means_4min_orbit_' + str(i)+'.pkl' 
    picklefile = open(filename,'rb')
    orbit_v_perps = pickle.load(picklefile)
    
    
        

    print('Combining input data dfs...')
    all_dates = orbit_means.index.union(fits_data.index)
    reindexed_orbit_means = orbit_means.reindex(all_dates)
    reindexed_fits_data = fits_data.reindex(all_dates)
    combined_current = reindexed_orbit_means.combine_first(reindexed_fits_data)
    beginning_of_orbit_data = orbit_means.index[0]
    end_of_orbit_data = orbit_means.index[-1]
    combined_updated_for_orbit = combined_current[beginning_of_orbit_data:end_of_orbit_data]
    combined_updated_for_orbit['Z_MAG'] = z_mag_info
    
    print('Grabbing compression data dfs for case study...')
    filename = '/home/mtfranciscovich/Juno-codes/compression_dfs_by_orbit/compression_df_orbit_'+str(i)+'.pkl'
    picklefile = open(filename,'rb')
    compression_df = pickle.load(picklefile)
    
    if do_case_study:  

        
    
        print('Calculating q...')
        q_datas = _get_q(do_case_study, case_study_dir, compression_df,True,True, combined_updated_for_orbit['Z_MAG'],orbit_v_perps,1.2e-2, 3e-1, 1e-2, 1.5e-1, combined_updated_for_orbit, combined_updated_for_orbit, interval, 1, False, True, False, i, 10, True,False, False)
        #                case_study, case_directory, compression_df,one_equation, theo_break, z_mag_df,delta_v_df,minf_index, maxf_index, minf, maxf, data_df, part_df, interval, dt, filter_magnitude, fits, moments,orbit_number, case_chance, test,empirical =False, remove_coi = False,acceptable_freqs_min = 2e-2, acceptable_freqs_max = 8e-2
        q_cleaned = q_datas[0][q_datas[0]['q'] != -1]
    else: 
        print('Calculating q...')
        q_datas = _get_q(do_case_study,case_study_dir, compression_df,True,True, combined_updated_for_orbit['Z_MAG'],orbit_v_perps,1.2e-2, 3e-1, 1e-2, 1.5e-1, combined_updated_for_orbit, combined_updated_for_orbit, interval, 1, False, True, False, i, 0, True,False, False)
        q_cleaned = q_datas[0][q_datas[0]['q'] != -1]
    
    if saving_pickles:
        q_datas[0].to_pickle(directory_to_store_orbit_qs+'/q_array_orbit_' +str(i)+'.pkl')
        df_extras_indexed = pd.DataFrame({'mhd_slopes': q_datas[1],
                                'kinetic_slopes': q_datas[2],
                                'R': q_datas[0]['mean_R'],
                                'Z': q_datas[0]['mean_Z'],
                                'local_times': q_datas[0]['local_times']},index = q_datas[0].index)
        df_extras_indexed.to_pickle(directory_to_store_q_pickles_extras_indexed+'/extras_indexed_orbit_' +str(i)+'.pkl')

        df_extras_not_indexed = pd.DataFrame({'breaks': q_datas[3],
                                'gyros': q_datas[4],
                                'thetas': q_datas[6]})


        df_extras_not_indexed.to_pickle(directory_to_store_q_pickles_extras_not_indexed+'/extras_not_indexed_orbit_' +str(i)+'.pkl')
        






In [ ]:
print(len(q_datas[1]))

print(len(q_datas[2]))

